# Exercise 1: 1D Automaton

We make a function that can simulate a 1D automaton with any rule (that uses two neighbours only)
We also make sure that it can do both Dirichlet (fixed) and periodic boundaries

In [ ]:
using Random
using Plots

"Random road with p occupation probabilty."
random_road(n::Int64, p::Float64) = Int8.(rand(n) .< p)

"Random road with exactly nCar number of cars."
random_road(n::Int64, nCar::Int64) = shuffle(vcat(zeros(Int8, n-nCar), ones(Int8, nCar)))

"""
Compute new state according to rule
- l,c,r are states of left, centre and right neighbours
- bitrule is the bit-wise rule as a string
"""
function compute_rule(l::Int8, c::Int8, r::Int8, bitrule::String)
    # Turn pattern into a 3-bit number, so we get a number 0 to 7.
    state = parse(Int, string(l, c, r), base = 2)

    # Now look-up number on our bitrule.
    return Int8(bitrule[end - state] == '1')
end

"""
Simulate a 1D automata with any rule.
Start with initialRoad and do nStep steps, using the given rule.
Boundaries can be :fixed or :periodic.
"""
function simulate_road(initialRoad::Array{Int8}, nStep::Int64, rule::Int64; 
                        boundaries = :fixed)
    
    # The rule is defined by 8 bits, so any integer between 0 and 2^8-1 can be a rule.
    rule > 255 && error("Rule needs to be an integer beteween 0 and 255")
    bitrule = bitstring(rule)[end-7:end]  # Transform integer into bitstring.
    @show bitrule
    
    @assert boundaries == :fixed || boundaries == :periodic "Boundaries need to be `:fixed` or `:periodic`"
    
    # Add padding on both sides of the road to account for boundary conditions.
    road2D = zeros(Int8, nStep, length(initialRoad) + 2)
    road2D[1,2:end-1] = initialRoad

    for i in 2:nStep
        if boundaries == :periodic
            road2D[i-1,1]   = road2D[i-1,end-1]
            road2D[i-1,end] = road2D[i-1,2]
        end
        
        road2D[i, 2:end-1] = compute_rule.(road2D[i - 1, 1:end-2], 
                                            road2D[i - 1, 2:end-1], 
                                            road2D[i - 1, 3:end], 
                                            bitrule)
    end
    
    return road2D
end

function plot_road(road2D::Matrix{Int8})
    heatmap(road2D[:,2:end-1], c = palette([:white, :red]), legend = :none)
    plot!(ylabel = "timestep")
    plot!(xlabel = "road position")
end

# Let's do a simple test with rule 184: two cars driving on a road.
Random.seed!(7)
roadLength = 5; nCar = 2; nStep = 20; rule = 184
boundaries = :periodic

road = random_road(roadLength,nCar)
road2D = simulate_road(road, nStep, rule, boundaries = boundaries)
plot_road(road2D)
# savefig("task1/1Dexample_$(String(boundaries)).pdf")

In [ ]:
Random.seed!(4)
roadLength = 40; probCar = .63; nStep = 50; rule = 184
road = random_road(roadLength,probCar) 

plot_road(simulate_road(road, nStep, rule, boundaries = :fixed))
plot!(title = "rule 184, p = $probCar")

# savefig("task1/1Dclass2_fixed_moreCars.pdf")

## Custom rules

In [ ]:
Random.seed!(42)
roadLength = 50; probCar = .4; nStep = 50
road = random_road(roadLength, probCar)

rules = [40, 56, 18, 110]
plots = [plot_road(simulate_road(road, nStep, rule, boundaries = :periodic)) for rule in rules]

plot(plots..., layout = (2, 2), legend = false)
[plot!(title = "rule $(rules[i])", subplot = i) for i=1:4]
plot!(size=(800,700))

# savefig("task1/1DcustomRules.pdf")

## Count moving cars

In [ ]:
function count_moving_cars(road2D::Matrix{Int8})
    # At each timestep, the number of moving cars is the number 
    # of occupied sites which were not occupied in the previous timestep.
    # We can calculate this as follows:
    nStep = size(road2D)[1]
    return sum(road2D[2:end, 2:end-1] .!= road2D[1:end-1, 2:end-1]) / 2(nStep-1)
end

# Plot an example to test the function.
road = random_road(50,5)
road2D = simulate_road(road, 20, 184, boundaries = :periodic)
println("average number of moving cars = ", count_moving_cars(road2D))
plot_road(road2D)

In [ ]:
Random.seed!(23)
nCars = range(1, 1000, step = 10)
roadLength = 1000
nStep = 1000

# Count moving cars for different number of cars in the system
movingCars = [count_moving_cars(simulate_road(random_road(roadLength, nCar), nStep, 184, boundaries = :periodic)) / nCar for nCar in nCars]


In [ ]:
using LaTeXStrings

scatter(nCars / roadLength, movingCars, label = "Simulation")
plot!(xlabel = "Occupation probability", ylabel = "Average number of moving cars")

xs = 0.5:1e-2:1
plot!(xs, (1 .- xs) ./ xs, label = L"\frac{1-p}{p}", linewidth = 2)
# savefig("task1/1DphaseTransition.pdf")

## Different occupation probabilities

In [ ]:
Random.seed!(42)
nStep = 100
roadLength = 300

ps = [.1, .35, .45, .55, .65, .8]
plots = [plot_road(simulate_road(random_road(roadLength, p), nStep, 184, boundaries = :periodic)) for p in ps]

plot(plots..., layout = (2, 3), legend = false)
[plot!(title = "p = $(ps[i])", subplot = i) for i=1:length(ps)]
plot!(size=(900,650))

# savefig("task1/1DdifferentPs.pdf")

## Make movie

In [ ]:
"Plots road as a circle."
function plot_circle_road(road; markersize = 10)
    L = length(road)
    
    # Find coordinates for "cars" in a circle centered at (0,0) with radius 1.
    # These are positions along the road.
    sites = [road[i] == 1 ? (cos(2π*i/L), sin(2π*i/L)) : (NaN, NaN) for i=1:L]
    
    scatter(sites, ratio = :equal, legend = :none, markersize = markersize, markercolor = :red)
    plot!(showaxis = false, ticks = false, aspect_ratio = :equal)
    plot!(xlims = (-1.1,1.1), ylims = (-1.1,1.1))
end

plot_circle_road([1,0,1,1,1,1,1,0])

In [ ]:
roadLength = 50
nCar = 32
nStep = 100
road = zeros(Int8, roadLength)
road[1:nCar] .= 1

road2D = simulate_road(road, nStep, 184, boundaries = :periodic)

skip = 1

mygif = @animate for s in 1:nStep
    plot_circle_road(road2D[s,2:end-1], markersize = 5)
    plot!(size = (200, 200))
    plot!(title = "timestep $s")
end

gif(mygif, "task1/1Dtraffic_nCar_$nCar.gif", fps = 10)

# Exercise 2: Game of Life

In [ ]:
"Computes new state of a site, given its current state and the sum of neighbours sumNeighbours"
function compute_rule(state::Int8, sumNeighbours::Int8)
    
    if sumNeighbours < 2
        return 0
    elseif sumNeighbours == 2
        return state
    elseif sumNeighbours == 3
        return 1
    elseif sumNeighbours > 3
        return 0
    end
end

"""
Returns the full game-of-life simulations as a 3D matrix.
- initialState of the domain in 2D.
- nStep is the number of steps to simulate
- boundaries can be :fixed or :periodic
"""
function game_of_life(initialState::Matrix{Int8}, nStep::Int64; boundaries = :fixed)
    n, m = size(initialState)
    
    @assert boundaries == :fixed || boundaries == :periodic "Boundaries need to be `:fixed` or `:periodic`"
    
    state3D = zeros(Int8, n+2, m+2, nStep)  # Add extra dimension for time, and padding cells around domain.
    state3D[2:end-1,2:end-1,1] = initialState
    
    for s in 2:nStep
        if boundaries == :periodic
            # Copy edges to have periodic boundaries.
            state3D[:,1,s-1] = state3D[:,end-1,s-1]
            state3D[:,end,s-1] = state3D[:,2,s-1]
            state3D[1,:,s-1] = state3D[end-1,:,s-1]
            state3D[end,:,s-1] = state3D[2,:,s-1]
            
            # Copy the corners.
            state3D[1,1,s-1] = state3D[end-1,end-1,s-1]
            state3D[end,1,s-1] = state3D[2,end-1,s-1]
            state3D[1,end,s-1] = state3D[end-1,2,s-1]
            state3D[end,end,s-1] = state3D[2,2,s-1]
        end
        
        for i in 2:n+1, j in 2:m+1
            state3D[i,j,s] = compute_rule(state3D[i,j,s-1], 
                state3D[i+1,j,s-1] +  # top
                state3D[i-1,j,s-1] +  # bottom
                state3D[i,j+1,s-1] +  # right
                state3D[i,j-1,s-1] +  # left
                state3D[i+1,j+1,s-1] +  # top-right
                state3D[i+1,j-1,s-1] +  # top-left
                state3D[i-1,j+1,s-1] +  # bottom-right
                state3D[i-1,j-1,s-1]  # bottom-left
                )
        end
    end
    
    return state3D 
end

"Place a blinker on matrix init, at position (i,j)"
function blinker!(init::Matrix{Int8}, i::Int64, j::Int64)
    init[i,j-1:j+1] .= 1
end

"Place a beehive on matrix init, at position (i,j)"
function beehive!(init::Matrix{Int8}, i::Int64, j::Int64)
    init[i,j:j+1] .= 1
    init[i-2,j:j+1] .= 1
    init[i-1,j-1] = 1
    init[i-1,j+2] = 1
end

"Place a glider on matrix init, at position (i,j)"
function glider!(init::Matrix{Int8}, i::Int64, j::Int64)
    init[i,j:j+2] .= 1
    init[i+1,j+2] = 1
    init[i+2,j+1] = 1
end

"Place a spaceship on matrix init, at position (i,j)"
function spaceship!(init::Matrix{Int8}, i::Int64, j::Int64)
    init[i,j:j+3] .= 1
    init[i+1:i+2,j+3] .= 1
    init[i+3,j+2] = 1
    init[i+1,j-1] = 1
    init[i+3,j-1] = 1
end


"Place a glider on matrix init, at position (x,y). The orientation of the glider can be 0, 1, 2, or 3."
function glider_gun!(system::Matrix{Int8}, x::Int64, y::Int64; rot = 0)
    
    # Make the gun.
    i,j = 4,1
    gun = zeros(9,36)
    gun[i:i+1,j:j+1] .= 1
    gun[i+2:i+3,j+34:j+35] .= 1
    
    gun[i-1:i+1, j+10] .= 1
    gun[i-2, j+11] = 1
    gun[i+2, j+11] = 1
    gun[i-3, j+12:j+13] .= 1
    gun[i+3, j+12:j+13] .= 1
    gun[i, j+14] = 1
    gun[i-2, j+15] = 1
    gun[i+2, j+15] = 1
    gun[i-1:i+1, j+16] .= 1
    gun[i, j+17] = 1
    
    gun[i+1:i+3, j+20:j+21] .= 1
    gun[i, j+22] = 1
    gun[i+4, j+22] = 1
    gun[i-1:i, j+24] .= 1
    gun[i+4:i+5, j+24] .= 1
    
    # Rotate gun.
    for i=1:rot
        gun = rotl90(gun)  # The function rotl90() rotates a matrix by 90 degrees
    end
    
    # Place gun on matrix.
    if rot == 0 || rot == 2
        init[x:x+35,y:y+8] = gun'
    elseif rot == 1 || rot == 3
        init[x:x+8, y:y+35] = gun'
    end 
end

In [ ]:
function do_nothing()
end

function animate_game_of_life(state3D::Array{Int8}; extraPlot::Function = do_nothing)
    nStep = size(state3D)[end]
    mygif = @animate for s in 1:nStep
        heatmap(state3D[2:end-1,2:end-1,s], c = palette([:darkblue, :red]), legend = :none)
        plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
        plot!(size = (500, 500))
        plot!(title = "Game of Life, step $s")
        extraPlot()
    end
    return mygif
end

In [ ]:
nStep = 200
init = zeros(Int8, 50,50)
blinker!(init,40,40)
blinker!(init,40,10)
beehive!(init,10,10)
glider!(init,20,10)
spaceship!(init,20,20)

state3D = game_of_life(init, nStep, boundaries = :periodic)

# heatmap(init, yflip = true)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, title = "initial state")

mygif = animate_game_of_life(state3D)

gif(mygif, "task2/game_of_life_periodic.gif", fps = 10)

In [ ]:
nStep = 200
init = zeros(Int8, 50,50)
glider!(init,20,10)
glider!(init,45,10)
glider!(init,20,10)
glider!(init,40,40)
spaceship!(init,20,20)
spaceship!(init,20,30)
spaceship!(init,30,40)

state3D = game_of_life(init, nStep, boundaries = :periodic)

mygif = animate_game_of_life(state3D)

gif(mygif, "task2/game_of_life_complex.gif", fps = 10)

In [ ]:
nStep = 200
Random.seed!(30)
init = Int8.(rand(50,50) .< 0.5)

state3D = game_of_life(init, nStep, boundaries = :periodic)

# heatmap(init, yflip = true)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, title = "initial state")

mygif = animate_game_of_life(state3D)

gif(mygif, "task2/game_of_life_random.gif", fps = 10)

In [ ]:
nStep = 200
init = zeros(Int8, 40,40)
glider!(init,20,10)
glider!(init,30,30)

state3D = game_of_life(init, nStep, boundaries = :periodic)

# heatmap(init, yflip = true)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, title = "initial state")

mygif = animate_game_of_life(state3D)

gif(mygif, "task2/gol_glider.gif", fps = 15)

In [ ]:
nStep = 300
init = zeros(Int8, 120,120)
glider_gun!(init,50,8, rot = 3)

state3D = game_of_life(init, nStep, boundaries = :fixed)

# heatmap(init, yflip = true)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, title = "initial state")

mygif = animate_game_of_life(state3D)

gif(mygif, "task2/gol_glider_gun.gif", fps = 28)

In [ ]:
nStep = 500
init = zeros(Int8, 100,100)
glider_gun!(init,56,36, rot = 2)
state3D = game_of_life(init, nStep, boundaries = :fixed)

# heatmap(init, yflip = true, size = (600,600))
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, title = "initial state")

function annotations()
    annotate!(65,35,"X", :white)
    annotate!(80,30,"Y", :white)
    plot!(title = "Y = NOT(X)")
end

mygif = animate_game_of_life(state3D, extraPlot = annotations)

gif(mygif, "task2/gol_not.gif", fps = 38)

In [ ]:
nStep = 500
init = zeros(Int8, 100,100)
glider_gun!(init,8,24, rot = 3)  # This glider gun represents X.
glider_gun!(init,56,36, rot = 2)
state3D = game_of_life(init, nStep, boundaries = :fixed)

mygif = animate_game_of_life(state3D, extraPlot = annotations)

gif(mygif, "task2/gol_not2.gif", fps = 38)

In [ ]:
nStep = 500
init = zeros(Int8, 100,100)
glider_gun!(init,8,24, rot = 3)
# glider_gun!(init,56,36, rot = 2)  # This glider gun represents X.
# glider_gun!(init,36,24, rot = 2)  # This glider gun represents Y.
state3D = game_of_life(init, nStep, boundaries = :fixed)

function annotations2()
    plot!(title = "Z = X AND Y")
    annotate!(40,40,"X", :white)
    annotate!(55,60,"Y", :white)
    annotate!(85,27,"Z", :white)
end

mygif = animate_game_of_life(state3D, extraPlot = annotations2)

gif(mygif, "task2/gol_and4.gif", fps = 38)

# Exercise 3: HPP gas model

In [ ]:
import Base.sum

"A site with up to 4 particles, each going in different direction."
struct Site
    u::Bool  # Up
    r::Bool  # Right
    d::Bool  # Down
    l::Bool  # Left
end

"Return total number of particles in site."
sum(s::Site) = s.u + s.r + s.d + s.l

"""
Returns reflected site along horizontal (axis = 0) or vertical (axis = 1)
"""
reflect(s::Site, axis::Int) = axis == 0 ? Site(s.u, s.l, s.d, s.r) : Site(s.d, s.r, s.u, s.l)

"Updates site by propagating neighbouring sites."
function propagate(site::Site, 
        usite::Site, rsite::Site, 
        dsite::Site, lsite::Site)
    return Site(dsite.u, lsite.r, usite.d, rsite.l)
end

"Updates site by taking into account collisions."
function collide(site::Site)
    if site == Site(1,0,1,0)
        return Site(0,1,0,1)
    elseif site == Site(0,1,0,1)
        return Site(1,0,1,0)
    end
    return site
end

"""
-initialState is a 2D matrix of Site elements with the initial gas distribution
-nStep is the number of steps to simulate
-boundaries can be :fixed, :periodic, or :reflecting

Returns a 3D array, where one dimension is time.
"""
function simulate_gas(initialState::Matrix{Site}, nStep::Int64; boundaries = :fixed)
    n,m = size(initialState)
    state3D = [Site(0,0,0,0) for i=1:n+2, j=1:m+2, s=1:nStep]  # Add padding sites around the domain.
    state3D[2:end-1,2:end-1,1] = initialState
    
    
    @assert boundaries == :fixed || boundaries == :periodic || boundaries == :reflecting "Boundaries need to be `:fixed`, `:periodic`, or `:reflecting`"

    for s = 2:nStep

        if boundaries == :periodic
            # Copy edges to have periodic boundaries
            state3D[:,1,s-1] = state3D[:,end-1,s-1]
            state3D[:,end,s-1] = state3D[:,2,s-1]
            state3D[1,:,s-1] = state3D[end-1,:,s-1]
            state3D[end,:,s-1] = state3D[2,:,s-1]
        elseif boundaries == :reflecting
            state3D[:,2,s-1] = reflect.(state3D[:,2,s-1], 0)
            state3D[:,end-1,s-1] = reflect.(state3D[:,end-1,s-1], 0)
            state3D[2,:,s-1] = reflect.(state3D[2,:,s-1], 1)
            state3D[end-1,:,s-1] = reflect.(state3D[end-1,:,s-1], 1)
        end
        
        state3D[2:end-1,2:end-1,s-1] = collide.(state3D[2:end-1,2:end-1,s-1])
        
        state3D[2:end-1,2:end-1,s] = propagate.(
            state3D[2:end-1,2:end-1,s-1],  # centre
            state3D[1:end-2,2:end-1,s-1],  # up neighbour
            state3D[2:end-1,3:end,s-1],  # right neighbour
            state3D[3:end,2:end-1,s-1],  # down neighbour
            state3D[2:end-1,1:end-2,s-1]  # left neighbour
            )
    end
    return state3D
end


function animate_gas_model(state3D::Array{Site}, title::String)
    nStep = size(state3D)[end]
    
    # We use a heatmap where darker sites contain more particles.
    mygif = @animate for s in 1:nStep
        heatmap(sum.(state3D[2:end-1, 2:end-1, s]), legend = :none, clim = (0,4), c = cgrad(:heat))
        plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
        plot!(size = (500, 500))
        plot!(title = "$title, step $s")
    end
    return mygif
end


# Test simple example:
nStep = 22
initialState = [Site(0,0,0,0) for i=1:20, j=1:20]
initialState[5,10] = Site(1,1,1,1)
initialState[15,10] = Site(1,1,1,1)
initialState[5,5] = Site(1,1,1,1)
initialState[15,5] = Site(1,1,1,1)
# initialState[7,7] = Site(1,1,1,1)
state3D = simulate_gas(initialState, nStep)

# Visualise initial conditions:
# heatmap(sum.(state3D[2:end-1, 2:end-1, 1]), legend = :none)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
# plot!(size = (500, 500))
# plot!(title = "Initial Gas state")

mygif = animate_gas_model(state3D, "HPP Gas Model, Dirichlet")
gif(mygif, "task3/HPP_model_simple.gif", fps = 4)


Dirichlet boundary conditions with empty padding-sites is equivalent to having open boundaries, i.e. particles exit the domain on the edges as if there was empty space.

In [ ]:
nStep = 50
initialState = [Site(0,0,0,0) for i=1:20, j=1:20]
initialState[5,9] = Site(1,1,1,1)
# initialState[15,5] = Site(1,1,1,1)

mygif = animate_gas_model(simulate_gas(initialState, nStep, boundaries = :periodic), "HPP Gas Model, periodic")

gif(mygif, "task3/HPP_model_simple_periodic.gif", fps = 12)

In [ ]:
nStep = 100
initialState = [Site(0,0,0,0) for i=1:20, j=1:20]
initialState[5,9] = Site(1,1,1,1)
# initialState[15,5] = Site(1,1,1,1)

mygif = animate_gas_model(simulate_gas(initialState, nStep, boundaries = :reflecting), "HPP Gas Model, reflecting")

gif(mygif, "task3/HPP_model_simple_reflecting.gif", fps = 12)

## Gaussian initial distribution

In [ ]:
using LinearAlgebra

# Make a Gaussian distribution:
Random.seed!(42)
μ = [35,35]
σ = 8
N = 70
nStep = 35

initGauss = Array{Site, 2}(undef,N,N)
for i=1:N, j=1:N
    p = exp(-norm([i,j]-μ)^2 / σ^2)
    initGauss[i,j] = Site(rand() < p, rand() < p, rand() < p, rand() < p)
end
initGauss[35,69] = Site(0,0,0,1)  # Adding an extra particle for fun.


boundaries = :fixed
mygif = animate_gas_model(simulate_gas(initGauss, nStep, boundaries = boundaries),
    "HPP Gas Model, $(String(boundaries))")

gif(mygif, "task3/HPP_model_gauss_$(String(boundaries)).gif", fps = 13)

In [ ]:
state3D = simulate_gas(initGauss, 35, boundaries = :fixed)
s = 30
heatmap(sum.(state3D[2:end-1, 2:end-1, s]), legend = :none, clim = (0,4), c = cgrad(:heat))
plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
plot!(size = (500, 500))
plot!(title = "HPP Gas Model, Dirichlet, step $s")

# savefig("task3/gas_model_fixed_step$s.pdf")

The problem with this method is that it always ends up in square symmetry, and cannot retain the initial circular symmetry.

## Hexagonal grid

In [ ]:
"Updates site by propagating neighbouring sites."
function propagate(neighbours::Vector{Int8}...)
    newsite = zeros(Int8, 6)
    newsite[1] = neighbours[4][1]
    newsite[2] = neighbours[5][2]
    newsite[3] = neighbours[6][3]
    newsite[4] = neighbours[1][4]
    newsite[5] = neighbours[2][5]
    newsite[6] = neighbours[3][6]
    return newsite
end

"Updates site by taking into account collisions."
function collide(site::Vector{Int8})
    s = sum(site)
    
    if s == 3
        if site == [1,0,1,0,1,0]
            newsite = [0,1,0,1,0,1]
        elseif site == [0,1,0,1,0,1]
            newsite = [1,0,1,0,1,0]
        else
            newsite = site
        end
        
    elseif s == 2
        if site == [1,0,0,1,0,0]
            newsite = rand() < 0.5 ? [0,1,0,0,1,0] : [0,0,1,0,0,1]
        elseif site == [0,1,0,0,1,0]
            newsite = rand() < 0.5 ? [1,0,0,1,0,0] : [0,0,1,0,0,1]
        elseif site == [0,0,1,0,0,1]
            newsite = rand() < 0.5 ? [0,1,0,0,1,0] : [1,0,0,1,0,0]
        else 
            newsite  = site
        end

    elseif s == 4
        if site == [1,1,0,1,1,0]
            newsite = rand() < 0.5 ? [0,1,1,0,1,1] : [1,0,1,1,0,1]
        elseif site == [1,0,1,1,0,1]
            newsite = rand() < 0.5 ? [0,1,1,0,1,1] : [1,1,0,1,1,0]
        elseif site == [0,1,1,0,1,1]
            newsite = rand() < 0.5 ? [1,1,0,1,1,0] : [1,0,1,1,0,1]
        else
            newsite = site
        end
    else
        newsite = site
    end

    return Int8.(newsite)
end


"""
-initialState is a 2D matrix of Site elements with the initial gas distribution
-nStep is the number of steps to simulate
-boundaries can be :fixed, :periodic, or :reflecting

Returns a 3D array, where one dimension is time.
"""
function simulate_gas(initialState::Matrix{Vector{Int8}}, nStep::Int64)
    n,m = size(initialState)
    state3D = [zeros(Int8, 6) for i=1:n+2, j=1:m+2, s=1:nStep]  # Add padding sites around the domain.
    state3D[2:end-1,2:end-1,1] = initialState
    
    for s = 2:nStep
        
        # Periodic boundary conditions by default.
        state3D[:,1,s-1] = state3D[:,end-1,s-1]
        state3D[:,end,s-1] = state3D[:,2,s-1]
        state3D[1,:,s-1] = state3D[end-1,:,s-1]
        state3D[end,:,s-1] = state3D[2,:,s-1]
        
        # Collision
        for i = 2:n+1
            for j = 2:m+1
                state3D[i,j,s-1] = collide(state3D[i,j,s-1])
            end
        end
        
        # Propagate. Needs to be done differently for even and odd rows.
        for i = 2:n+1
            for j = 2:m+1 
                if i%2 == 0
                    state3D[i,j,s] = propagate(
                        state3D[i-1,j,s-1], state3D[i-1,j+1,s-1], # Above neigbours
                        state3D[i,j+1,s-1], # Right n
                        state3D[i+1,j+1,s-1], state3D[i+1,j,s-1], # Below nei
                        state3D[i,j-1,s-1])
                else
                    state3D[i,j,s] = propagate(
                        state3D[i-1,j-1,s-1], state3D[i-1,j,s-1], # Above neigbours
                        state3D[i,j+1,s-1], # Right n
                        state3D[i+1,j,s-1], state3D[i+1,j-1,s-1], # Below nei
                        state3D[i,j-1,s-1])
                end
            end
        end
        
    end
    return state3D
end

function animate_gas_model(state3D::Array{Vector{Int8}}, title::String; markersize = 10)
    n,m,nStep = size(state3D)
    myColors = [cgrad(:Wistia, [0.01, 0.99])[z] for z ∈ range(0.0, 1.0, length = 6)]
    
    # We use a heatmap where darker sites contain more particles.
    mygif = @animate for s in 1:nStep
        xs, ys, cs = [],[],[]
        for i = 2:n-1
            for j = 2:m-1
                if state3D[i,j,s] != zeros(Int8, 6)
                    push!(xs, i)
                    push!(ys, i%2==0 ? j : j - .5)
                    push!(cs, sum(state3D[i,j,s]))
                end
            end
        end
        
        scatter(xs, ys, markersize = markersize, color = myColors[cs], legend = :none)
        plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
        plot!(size = (500, 500))
        plot!(xlims = (0,m), ylims = (0,n))
        plot!(title = "$title, step $s")
    end
    return mygif
end

# Small test
nStep = 10
initialState = [zeros(Int8, 6) for i=1:20, j=1:20]
initialState[5,10] = Int8.([0,0,1,0,0,0])
initialState[5,14] = Int8.([0,0,0,0,0,1])

state3D = simulate_gas(initialState, nStep)

# Visualise initial conditions:
# heatmap(sum.(state3D[2:end-1, 2:end-1, 1]), legend = :none)
# plot!(showaxis = false, ticks = false, aspect_ratio = :equal, yflip = true)
# plot!(size = (500, 500))
# plot!(title = "Initial Gas state")

mygif = animate_gas_model(state3D, "FHP Gas Model, Periodic", markersize = 10)
gif(mygif, "task3/FHP_model_simple.gif", fps = 2)

In [ ]:
# Make a Gaussian distribution:
Random.seed!(42)
μ = [100,100]
σ = 50
N = 200
nStep = 200

initGauss = Array{Vector{Int8}, 2}(undef,N,N)
for i=1:N, j=1:N
    p = exp(-norm([i,j]-μ)^2 / σ^2)
    initGauss[i,j] = [rand() < p ? Int8(1) : Int8(0) for i = 1:6]
end
initGauss[32,2] = Int8.([0,0,1,0,0,0])  # Add another particle for fun.

mygif = animate_gas_model(simulate_gas(initGauss, nStep), "FHP Gas Model, Periodic", markersize = 3)
gif(mygif, "task3/FHP_model_gauss.gif", fps = 12)